# Analyzing scores and plotting them

In [3]:
import json
import pandas as pd

## Inspect scores
roles_20

In [4]:
# load scores from roles_20 as pandas dataframe

roles_path = "/root/git/persona-subspace/evals/results/roles_traits/roles_20_scores.jsonl"
default_path = "/root/git/persona-subspace/evals/results/roles_traits/default_20_scores.jsonl"
harmbench_path = "/root/git/persona-subspace/evals/results/roles_traits/harmbench_scores.jsonl"

In [5]:
# Load all 3 JSONL files into separate DataFrames
with open(roles_path, "r") as f:
    roles_scores = [json.loads(line) for line in f]
    
with open(default_path, "r") as f:
    default_scores = [json.loads(line) for line in f]
    
with open(harmbench_path, "r") as f:
    harmbench_scores = [json.loads(line) for line in f]

# Convert to pandas DataFrames
roles_df = pd.DataFrame(roles_scores)
default_df = pd.DataFrame(default_scores)
harmbench_df = pd.DataFrame(harmbench_scores)

print(f"Loaded {len(roles_df)} roles scores")
print(f"Loaded {len(default_df)} default scores") 
print(f"Loaded {len(harmbench_df)} harmbench scores")

Loaded 3000 roles scores
Loaded 3000 default scores
Loaded 2862 harmbench scores


In [6]:
# Distribution of roles_20 scores by magnitude
print("=== Roles_20 Score Distribution by Magnitude ===")
print(f"Total samples: {len(roles_df)}")
print()
grouped = roles_df.groupby('magnitude')['score'].value_counts().sort_index()
print(grouped.to_string())


=== Roles_20 Score Distribution by Magnitude ===
Total samples: 3000

magnitude  score
-4000.0    0        142
           1        851
           2          6
           3          1
-2000.0    0          4
           1        842
           2         32
           3        122
 0.0       1          3
           3        997


In [10]:
# print 10 prompts and responses from roles_df from each magnitude x score combination
print(roles_df.groupby(['magnitude', 'score']).apply(lambda x: x.sample(min(len(x), 10))[['prompt', 'response']]))

                                                                 prompt  \
magnitude score                                                           
-4000.0   0     1132  Embody an eternal dragon whose vast experience...   
                1104  You are an ancient dragon of immense wisdom an...   
                833   Take on the role of a sphinx that guards ancie...   
                834   Take on the role of a sphinx that guards ancie...   
                346   Take on the role of a whimsical yet dangerous ...   
...                                                                 ...   
 0.0      3     2153  You are a charming Southern belle with graciou...   
                282   Embody the role of a courteous gentleman who e...   
                1815  Act as a livestream moderator with deep knowle...   
                1937  You are a contemplative monk who approaches al...   
                1822  Please be a livestream moderator who specializ...   

                        

/tmp/ipykernel_4131101/1820822918.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(roles_df.groupby(['magnitude', 'score']).apply(lambda x: x.sample(min(len(x), 10))[['prompt', 'response']]))


In [15]:
# Distribution of default_20 scores by magnitude
print("=== Default_20 Score Distribution by Magnitude ===")
print(f"Total samples: {len(default_df)}")
print()
grouped = default_df.groupby('magnitude')['score'].value_counts().sort_index()
print(grouped.to_string())


=== Default_20 Score Distribution by Magnitude ===
Total samples: 3000

magnitude  score
0.0        0         112
           1         593
           2         244
           3          51
2000.0     2         167
           3         833
4000.0     3        1000


In [16]:
# Distribution of harmbench scores by magnitude
print("=== Harmbench Score Distribution by Magnitude ===")
print(f"Total samples: {len(harmbench_df)}")
print()
grouped = harmbench_df.groupby('magnitude')['score'].value_counts().sort_index()
print(grouped.to_string())


=== Harmbench Score Distribution by Magnitude ===
Total samples: 2862

magnitude  score
0.0        no       845
           yes      109
2000.0     no       639
           yes      315
4000.0     no       833
           yes      121
